In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import ttk, messagebox

In [2]:
def get_user_inputs():
    num_assets = int(asset_spinbox.get())
    if not 2 <= num_assets <= 12:
        raise ValueError("Number of assets must be between 2 and 12.")
    
    ex_returns_values = [float(val) for val in ex_returns_entry.get().split(',')]
    vol_values = [float(val) for val in vol_entry.get().split(',')]
    corr_values = [float(val) for val in corr_entry.get().split(',')]
    risk_adv = float(risk_adv_entry.get())
    risk_free = float(risk_free_entry.get())
    
    if len(ex_returns_values) != num_assets or len(vol_values) != num_assets or len(corr_values) != num_assets**2:
        raise ValueError("Incorrect number of inputs for returns, volatilities, or correlation coefficients.")
    
    return num_assets, ex_returns_values, vol_values, corr_values, risk_adv, risk_free

In [3]:
def load_data(num_assets, desired_tickers):
    df = pd.read_csv('/Users/ianjonsson/Desktop/Python CW.csv', header=0)
    df = df.dropna(axis=1)
    df = df.drop(columns=["Date"])
    df = df[desired_tickers]
    return df

In [4]:
def calculate_matrices(df, ex_returns_values, vol_values, corr_values, desired_tickers):
    ex_returns = pd.Series(ex_returns_values, index=desired_tickers)
    vol = pd.Series(vol_values, index=desired_tickers)
    corr_matrix = np.array(corr_values).reshape((len(desired_tickers), len(desired_tickers)))
    corr = pd.DataFrame(corr_matrix, index=desired_tickers, columns=desired_tickers)
    cov = corr.mul(vol, axis=0).mul(vol, axis=1)
    inv_cov = np.linalg.inv(cov)
    return ex_returns, cov, inv_cov

In [5]:
def calculate_weights(ex_returns, cov, inv_cov):
    l_matrix = ex_returns @ inv_cov
    vector = pd.Series([1] * len(ex_returns))
    m_matrix = vector @ inv_cov

    a = vector @ l_matrix
    b = ex_returns @ l_matrix
    c = vector @ m_matrix
    d = b * c - (a ** 2)

    g = 1/(d) * ((m_matrix*b) - (l_matrix*a))
    h = 1/(d) * ((l_matrix*c) - (m_matrix*a))
    gh = g + h

    return g, h, gh, a, b, c, d

In [6]:
def run_analysis():
    try:
        num_assets, ex_returns_values, vol_values, corr_values, risk_adv, risk_free = get_user_inputs()

        tickers = {
            # 1: "Artemis Corporate Bond",
            # 2: "Royal London Corporate Bond",
            # 3: "Legal & General All Stocks Gilt Index Trust",
            # 4: "iShares Corporate Bond Index",
            # 5: "JPM Emerging Markets",
            # 6: "CT European Select",
            # 7: "Barings Europe Select",
            # 8: "Polar Capital European ex-UK Income",
            # 9: "BlackRock Continental European Income",
            # 10: "Legal & General European Index",
            # 11: "Rathbone Global Opportunities",
            # 12: "Baillie Gifford Managed",
            1: "Nvidia",
            2: "AAPL",
            3: "BAESY",
            4: "CJJD",
            5: "AZN",
            6: "SAS",
            7: "TSLA",
            8: "Saab AB",
            9: "PFE",
            10: "GME",
            11: "HSBC",
            12: "LVMH"
        }

        desired_columns = list(range(1, num_assets + 1))
        desired_tickers = [tickers[i] for i in desired_columns]

        df = load_data(num_assets, desired_tickers)
        ex_returns, cov, inv_cov = calculate_matrices(df, ex_returns_values, vol_values, corr_values, desired_tickers)
        g, h, gh, a, b, c, d = calculate_weights(ex_returns, cov, inv_cov)

        optimum_weights = g + (target_return * h)

        print("l_matrix:", ex_returns @ inv_cov)
        print("m_matrix:", pd.Series([1] * len(ex_returns)) @ inv_cov)
        print("a:", a)
        print("b:", b)
        print("c:", c)
        print("d:", d)
        print("g:", g)
        print("h:", h)
        print("gh:", gh)
        print("optimum_weights:", optimum_weights)

        return_optimum_point = optimum_weights @ ex_returns
        risk_optimum_point = (optimum_weights.T @ cov @ optimum_weights) ** 0.5

        print("return_optimum_point:", return_optimum_point)
        print("risk_optimum_point:", risk_optimum_point)

        return_g = g @ ex_returns
        risk_g = (g.T @ cov @ g) ** 0.5

        return_h = h @ ex_returns
        risk_h = (h.T @ cov @ h) ** 0.5

        return_gh = gh @ ex_returns
        risk_gh = (gh.T @ cov @ gh) ** 0.5

        print(f"risk_g={risk_g}\nrisk_h={risk_h}\nrisk_g+h={risk_gh}")

        min_var = a/c
        opt_var = min_var - (d/(c**2))/(risk_free - min_var)

        print("min_var:", min_var)
        print("opt_var:", opt_var)

        portfolio_returns = []
        portfolio_risk = []
        portfolio_weights = []
        portfolio_utility = []
        portfolio_sharpe_ratios = []

        p_returns = np.linspace(0, 1, 101)

        for i in p_returns:
            p_weights = g + (i * h)
            p_return = np.sum(p_weights * ex_returns)
            portfolio_variance = np.dot(p_weights.T, np.dot(cov, p_weights))
            p_risk = (p_weights @ cov @ p_weights.T) ** 0.5
            excess_return = p_return - risk_free
            p_sharpe_ratios = excess_return / p_risk
            p_utility = p_return - (0.5 * risk_adv * portfolio_variance)

            portfolio_weights.append(p_weights)
            portfolio_risk.append(p_risk)
            portfolio_utility.append(p_utility)
            portfolio_sharpe_ratios.append(p_sharpe_ratios)
            portfolio_returns.append(p_return)

        portfolio_returns = np.array(portfolio_returns)
        portfolio_risk = np.array(portfolio_risk)
        portfolio_weights = np.array(portfolio_weights)
        portfolio_utility = np.array(portfolio_utility)

        max_sharpe = max(portfolio_sharpe_ratios)

        portfolio_capital_allocations = []

        for i in p_returns:
            p_weights = g + (i * h)
            p_risk = (p_weights @ cov @ p_weights.T) ** 0.5
            p_capital_allocation = risk_free + (max_sharpe * p_risk)
            portfolio_capital_allocations.append(p_capital_allocation)

        portfolio_capital_allocations = np.array(portfolio_capital_allocations)

        weight_columns = [f'{desired_tickers[i]} Weight' for i in range(len(desired_columns))]
        data = {
            'Return': [p_ret for p_ret in portfolio_returns],
            'Volatility': [risk for risk in portfolio_risk],
            'Sharpe Ratio': [sharpe for sharpe in portfolio_sharpe_ratios],
            'Utility': [utility for utility in portfolio_utility],
            'Capital Allocation Line Ret': [capital for capital in portfolio_capital_allocations]
        }

        for i, col in enumerate(weight_columns):
            data[col] = [w[i] for w in portfolio_weights]

        df = pd.DataFrame(data)

        df["Capital Allocation Line Ret"] = risk_free + (max_sharpe * df["Volatility"])

        # Define the path to save the Excel file on the desktop
        excel_file = '/Users/ianjonsson/Desktop/portfolio_analysis.xlsx'
        df.to_excel(excel_file, index=False)
        messagebox.showinfo("Success", f"Data saved to {excel_file}")

        plt.figure(figsize=(10, 6))
        plt.plot(portfolio_risk, portfolio_returns)
        plt.title('Efficient Frontier')
        plt.xlabel('Risk (Standard Deviation)')
        plt.ylabel('Return')
        plt.show()

    except Exception as e:
        messagebox.showerror("Error", str(e))

In [7]:
# Create the main window
root = Tk()
root.title("Asset Selection")

''

In [8]:
# Add a label and a spinbox for the number of assets
label = Label(root, text="Select number of assets (2-12):")
label.pack(pady=10)

asset_spinbox = ttk.Spinbox(root, from_=2, to=12, increment=1)
asset_spinbox.pack(pady=10)

In [9]:
# Add entries for expected returns, volatility, correlation, risk aversion, and risk-free rate
ex_returns_label = Label(root, text="Expected Rates of Return (comma separated):")
ex_returns_label.pack(pady=5)
ex_returns_entry = Entry(root, width=50)
ex_returns_entry.pack(pady=5)

vol_label = Label(root, text="Volatility (comma separated):")
vol_label.pack(pady=5)
vol_entry = Entry(root, width=50)
vol_entry.pack(pady=5)

corr_label = Label(root, text="Correlation Coefficients (comma separated, row-wise):")
corr_label.pack(pady=5)
corr_entry = Entry(root, width=50)
corr_entry.pack(pady=5)

risk_adv_label = Label(root, text="Risk Aversion Coefficient:")
risk_adv_label.pack(pady=5)
risk_adv_entry = Entry(root, width=50)
risk_adv_entry.pack(pady=5)

risk_free_label = Label(root, text="Risk-Free Rate of Return:")
risk_free_label.pack(pady=5)
risk_free_entry = Entry(root, width=50)
risk_free_entry.pack(pady=5)

In [10]:
# Add a button to run the analysis
button = Button(root, text="Run Analysis", command=run_analysis)
button.pack(pady=20)

In [11]:
# Run the Tkinter event loop
root.mainloop()